# UTS

## Klasifikasi merupakan kata serapan dari bahasa Belanda, classificatie, yang sendirinya berasal dari bahasa Prancis classification.


## Load Data

In [31]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [32]:
%cd /content/drive/My Drive/prosaindata/

/content/drive/My Drive/prosaindata


In [33]:
#install library
!pip install sastrawi
!pip install swifter
!pip install gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [34]:
#import library
import pandas as pd
import numpy as np
from string import punctuation
import re
import nltk

In [35]:
#load data
df = pd.read_excel("tugas/dataset/Data_TA.xlsx")
df.head()

,NPM,Judul,Abstrak,Prodi,Label
0,40411100468,PERANCANGAN DAN IMPLEMENTASI SISTEM DATABASE T...,Sistem informasi akademik (SIAKAD) merupaka...,Teknik Informatika,RPL
1,40411100476,APLIKASI KONTROL DAN MONITORING JARINGAN KOMPU...,Berjalannya koneksi jaringan komputer dengan l...,Teknik Informatika,RPL
2,40411100480,RANCANG BANGUN APLIKASI PROXY SERVER UNTUKENKR...,Web server adalah sebuah perangkat lunak serve...,Teknik Informatika,RPL
3,70411100070,SISTEM PENDUKUNG KEPUTUSAN OPTIMASI PENJADWALA...,Penjadwalan kuliah di Perguruan Tinggi me...,Teknik Informatika,komputasi
4,80411100115,SISTEM AUGMENTED REALITY ANIMASI BENDA BERGERA...,Seiring perkembangan teknologi yang ada diduni...,Teknik Informatika,komputasi


In [36]:
df.shape

(853, 5)

In [37]:
df.isnull().sum()

NPM         0
Judul       6
Abstrak    25
Prodi       0
Label      43
dtype: int64

In [38]:
df = df.dropna()

In [39]:
df.shape

(810, 5)

In [40]:
df.isnull().sum()

NPM        0
Judul      0
Abstrak    0
Prodi      0
Label      0
dtype: int64

In [41]:
df['Label'].value_counts()

komputasi    573
RPL          198
PBA           39
Name: Label, dtype: int64

## Preprocessing

### 1. Symbol & Punctuation Removal, case folding

In [42]:
#proses menghilangkan simbol dan emoji
def remove_text_special (text):
  text = text.replace('\\t',"").replace('\\n',"").replace('\\u',"").replace('\\',"")
  text = text.encode('ascii', 'replace').decode('ascii')
  return text.replace("http://"," ").replace("https://", " ")
df['Abstrak'] = df['Abstrak'].apply(remove_text_special)
print(df['Abstrak'])

0      Sistem  informasi  akademik  (SIAKAD) merupaka...
1      Berjalannya koneksi jaringan komputer dengan l...
2      Web server adalah sebuah perangkat lunak serve...
3      Penjadwalan  kuliah  di  Perguruan  Tinggi  me...
4      Seiring perkembangan teknologi yang ada diduni...
                             ...                        
848    Investasi saham selama ini memiliki resiko ker...
849    Information Retrieval (IR) merupakan pengambil...
850    Klasifikasi citra merupakan proses pengelompok...
851    Identifikasi atribut pejalan kaki merupakan sa...
852    Topik deteksi objek telah menarik perhatian ya...
Name: Abstrak, Length: 810, dtype: object


<ipython-input-42-bebc9197bc1f>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Abstrak'] = df['Abstrak'].apply(remove_text_special)


In [43]:
def remove_tanda_baca(text):
  text = re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",text)
  return text

df['Abstrak'] = df['Abstrak'].apply(remove_tanda_baca)
df['Abstrak'].head(20)

0     Sistem  informasi  akademik   SIAKAD  merupaka...
1     Berjalannya koneksi jaringan komputer dengan l...
2     Web server adalah sebuah perangkat lunak serve...
3     Penjadwalan  kuliah  di  Perguruan  Tinggi  me...
4     Seiring perkembangan teknologi yang ada diduni...
5     Gerak pekerja ada pada game yang memiliki genr...
6     Perkembangan game yang semakin pesat  memberik...
7     Sistem pengenalan wajah adalah suatu sistem un...
8     Teknologi mobile game beroperating system open...
9     Kantor Badan Kepegawaian kota Bangkalan adalah...
10    Penyusunan Sistem Informasi Dinas Perindustria...
11    Perusahaan pemerintah maupun swasta mempunyai ...
12    Pelayanan makanan bagi pasien rawat inap di Ru...
13    Penyusunan Sistem Pendukung Keputusan pemiliha...
14    Sidik jari adalah salah satu karakteristik fis...
15    Di Indonesia masalah perkembangan gizi adalah ...
16    Pengenalan tulisan tangan merupakan topik pene...
17    Citra senyum merupakan salah satu fitur bi

In [44]:
#proses menghilangkan angka
def remove_numbers (text):
  return re.sub(r"\d+", "", text)
df['Abstrak'] = df['Abstrak'].apply(remove_numbers)
df['Abstrak']

0      Sistem  informasi  akademik   SIAKAD  merupaka...
1      Berjalannya koneksi jaringan komputer dengan l...
2      Web server adalah sebuah perangkat lunak serve...
3      Penjadwalan  kuliah  di  Perguruan  Tinggi  me...
4      Seiring perkembangan teknologi yang ada diduni...
                             ...                        
848    Investasi saham selama ini memiliki resiko ker...
849    Information Retrieval  IR  merupakan pengambil...
850    Klasifikasi citra merupakan proses pengelompok...
851    Identifikasi atribut pejalan kaki merupakan sa...
852    Topik deteksi objek telah menarik perhatian ya...
Name: Abstrak, Length: 810, dtype: object

In [45]:
#proses casefolding
def casefolding(Comment):
  Comment = Comment.lower()
  return Comment
df['Abstrak'] = df['Abstrak'].apply(casefolding)
df['Abstrak']

0      sistem  informasi  akademik   siakad  merupaka...
1      berjalannya koneksi jaringan komputer dengan l...
2      web server adalah sebuah perangkat lunak serve...
3      penjadwalan  kuliah  di  perguruan  tinggi  me...
4      seiring perkembangan teknologi yang ada diduni...
                             ...                        
848    investasi saham selama ini memiliki resiko ker...
849    information retrieval  ir  merupakan pengambil...
850    klasifikasi citra merupakan proses pengelompok...
851    identifikasi atribut pejalan kaki merupakan sa...
852    topik deteksi objek telah menarik perhatian ya...
Name: Abstrak, Length: 810, dtype: object

### 2. Word Normalization

In [46]:
#proses tokenisasi
# from nltk.tokenize import TweetTokenizer
nltk.download('punkt')
# def word_tokenize(text):
#   tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
#   return tokenizer.tokenize(text)

df['review_token'] = df['Abstrak'].apply(lambda sentence: nltk.word_tokenize(sentence))
df['review_token']

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


0      [sistem, informasi, akademik, siakad, merupaka...
1      [berjalannya, koneksi, jaringan, komputer, den...
2      [web, server, adalah, sebuah, perangkat, lunak...
3      [penjadwalan, kuliah, di, perguruan, tinggi, m...
4      [seiring, perkembangan, teknologi, yang, ada, ...
                             ...                        
848    [investasi, saham, selama, ini, memiliki, resi...
849    [information, retrieval, ir, merupakan, pengam...
850    [klasifikasi, citra, merupakan, proses, pengel...
851    [identifikasi, atribut, pejalan, kaki, merupak...
852    [topik, deteksi, objek, telah, menarik, perhat...
Name: review_token, Length: 810, dtype: object

In [47]:
#Normalisasi kata tidak baku
normalize = pd.read_excel("tugas/dataset/Normalization Data.xlsx")

normalize_word_dict = {}

for row in normalize.iterrows():
  if row[0] not in normalize_word_dict:
    normalize_word_dict[row[0]] = row[1]

def normalized_term(comment):
  return [normalize_word_dict[term] if term in normalize_word_dict else term for term in comment]

df['comment_normalize'] = df['review_token'].apply(normalized_term)
df['comment_normalize'].head(20)

0     [sistem, informasi, akademik, siakad, merupaka...
1     [berjalannya, koneksi, jaringan, komputer, den...
2     [web, server, adalah, sebuah, perangkat, lunak...
3     [penjadwalan, kuliah, di, perguruan, tinggi, m...
4     [seiring, perkembangan, teknologi, yang, ada, ...
5     [gerak, pekerja, ada, pada, game, yang, memili...
6     [perkembangan, game, yang, semakin, pesat, mem...
7     [sistem, pengenalan, wajah, adalah, suatu, sis...
8     [teknologi, mobile, game, beroperating, system...
9     [kantor, badan, kepegawaian, kota, bangkalan, ...
10    [penyusunan, sistem, informasi, dinas, perindu...
11    [perusahaan, pemerintah, maupun, swasta, mempu...
12    [pelayanan, makanan, bagi, pasien, rawat, inap...
13    [penyusunan, sistem, pendukung, keputusan, pem...
14    [sidik, jari, adalah, salah, satu, karakterist...
15    [di, indonesia, masalah, perkembangan, gizi, a...
16    [pengenalan, tulisan, tangan, merupakan, topik...
17    [citra, senyum, merupakan, salah, satu, fi

### 3. Tokenizing

### 4. Stopwords Removal

In [48]:
#Stopword Removal
nltk.download('stopwords')
from nltk.corpus import stopwords
txt_stopwords = stopwords.words('indonesian')

def stopwords_removal(filtering) :
  filtering = [word for word in filtering if word not in txt_stopwords]
  return filtering

df['stopwords_removal'] = df['comment_normalize'].apply(stopwords_removal)
df['stopwords_removal'].head(20)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


0     [sistem, informasi, akademik, siakad, sistem, ...
1     [berjalannya, koneksi, jaringan, komputer, lan...
2     [web, server, perangkat, lunak, server, berfun...
3     [penjadwalan, kuliah, perguruan, kompleks, per...
4     [seiring, perkembangan, teknologi, didunia, mu...
5     [gerak, pekerja, game, memiliki, genre, rts, r...
6     [perkembangan, game, pesat, alternative, pemin...
7     [sistem, pengenalan, wajah, sistem, mengenali,...
8     [teknologi, mobile, game, beroperating, system...
9     [kantor, badan, kepegawaian, kota, bangkalan, ...
10    [penyusunan, sistem, informasi, dinas, perindu...
11    [perusahaan, pemerintah, swasta, tujuan, dicap...
12    [pelayanan, makanan, pasien, rawat, inap, ruma...
13    [penyusunan, sistem, pendukung, keputusan, pem...
14    [sidik, jari, salah, karakteristik, fisik, man...
15    [indonesia, perkembangan, gizi, yangperlu, per...
16    [pengenalan, tulisan, tangan, topik, penelitia...
17    [citra, senyum, salah, fitur, biometrik, d

In [49]:
#stopword removal 2
data_stopwords = pd.read_excel("tugas/dataset/list_stopwords.xlsx")
print(data_stopwords)

def stopwords_removal2(filter) :
  filter = [word for word in filter if word not in data_stopwords]
  return filter

df['stopwords_removal_final'] = df['stopwords_removal'].apply(stopwords_removal2)
df['stopwords_removal_final'].head(20)

Empty DataFrame
Columns: [kah, samping, kiri, maju, mundur, orang kedua, youtube, nya, non, gaada, hehe, heheee, haha, bf, dll, un, safa, ida, idak, aja, kk, wkwkk, gimana, lihat, duel, boong, ya, coba, anjing, pilihan, sandi, seklai, anggi, tetima, ubtik, tai, t, f, a, v, lt, cq, fd, e, h, da, diba, ax, babu, tombol, kantuk, kntl, sherina, ter, eh, kitaudah, cari, daan, ketemu, sdang, luring, pas, luring.1, bilang, log, in, kembaliterimah, nge, lo, dakwahlbh, klan, detik, tolol, kaka, kakk, kakak, silah, mbaaak, mbak, kn, fasid, solo, loh, bagusbagus, an, nan, babi, negara, m, t.1, o, p, ii, nyacuma, putarpilihdengerinklo, sia, cv, katai, luring.2, log.1, ...]
Index: []

[0 rows x 357 columns]


0     [sistem, informasi, akademik, siakad, sistem, ...
1     [berjalannya, koneksi, jaringan, komputer, lan...
2     [web, server, perangkat, lunak, server, berfun...
3     [penjadwalan, kuliah, perguruan, kompleks, per...
4     [seiring, perkembangan, teknologi, didunia, mu...
5     [gerak, pekerja, memiliki, genre, rts, real, t...
6     [perkembangan, pesat, alternative, peminatnya,...
7     [sistem, pengenalan, wajah, sistem, mengenali,...
8     [teknologi, mobile, beroperating, system, open...
9     [kantor, badan, kepegawaian, kota, bangkalan, ...
10    [penyusunan, sistem, informasi, dinas, perindu...
11    [perusahaan, pemerintah, swasta, tujuan, dicap...
12    [pelayanan, makanan, pasien, rawat, inap, saki...
13    [penyusunan, sistem, pendukung, keputusan, pem...
14    [sidik, jari, salah, karakteristik, fisik, man...
15    [indonesia, perkembangan, gizi, yangperlu, per...
16    [pengenalan, tulisan, tangan, topik, penelitia...
17    [citra, senyum, salah, fitur, biometrik, d

### 5. Stemming

In [50]:
#proses stem
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import string
import swifter
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stemming (term):
  return stemmer.stem(term)

term_dict = {}
for document in df['stopwords_removal_final']:
  for term in document:
    if term not in term_dict:
      term_dict[term] = ''


In [51]:
print(len(term_dict))
print("-----------------------------")

8482
-----------------------------


In [ ]:
for term in term_dict:
  term_dict[term] = stemming(term)
  print(term,":",term_dict[term])

print(term_dict)
print("-----------------------------")

Streaming output truncated to the last 5000 lines.
share : share
terhubung : hubung
gps : gps
tracking : tracking
dipasang : pasang
kemanfaatan : manfaat
pemainan : main
alfagram : alfagram
alfabetis : alfabetis
cinta : cinta
tinca : tinca
mengetik : etik
perhuruf : huruf
rancangan : rancang
frame : frame
markers : markers
penanda : tanda
susun : susun
kuantitatif : kuantitatif
puas : puas
krisis : krisis
fire : fire
institute : institute
pmi : pmi
risk : risk
risiko : risiko
merespon : merespon
cost : cost
scope : scope
sap : sap
semen : semen
gresik : gresik
tbk : tbk
sale : sale
serangkaian : rangkai
mahkota : mahkota
qr : qr
quick : quick
response : response
pengkodean : kode
denso : denso
barcode : barcode
pembacaan : baca
cahaya : cahaya
makhluk : makhluk
removal : removal
thresholding : thresholding
q : q
sequence : sequence
generasi : generasi
muda : muda
tuntutan : tuntut
kemendikbud : kemendikbud
mengeluarkan : keluar
kurikulum : kurikulum
ajaran : ajar
peminatan : minat
angk

In [ ]:
def get_stemming(document):
  return [term_dict[term] for term in document]

In [ ]:
df['stemming'] = df['stopwords_removal_final'].swifter.apply(get_stemming)

In [ ]:
print(df['stemming'])

In [ ]:
df.head(20)

## PCA

**Principal Component Analysis (PCA)** adalah salah satu metode reduksi dimensi pada machine learning. PCA akan memilih œvariabel-variabel yang mampu menjelaskan sebagian besar variabilitas data. PCA mengurangi dimensi dengan membentuk variabel-variabel baru yang disebut Principal Components. Principal Components yang merupakan kombinasi linier dari variabel-variabel lama. Penghitungan Varians dan Principal Component ini dapat dilakukan dengan menggunakan konsep nilai eigen (eigenvalue) dan vektor eigen (eigenvector) dari ilmu Aljabar Linier.

**Principal Component Analysis (PCA)** tentunya punya banyak manfaat dalam proses analisis data, misal:

*   Mengatasi multikolinieritas yang pada metode parametrik tertentu merupakan 
*   asumsi yang harus dipenuhi
*   Mereduksi jumlah variabel yang akan dimasukkan ke model
*   Jumlah variabel yang lebih sedikit tentu akan menyederhanakan model
*   Juga mempercepat proses komputasi

### Normalisasi Data (Standart Scaler)

In [ ]:
X = df.loc[:,'stemming']
Y = df['Label']
print(X)

In [ ]:
Y

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
scaler = sc.fit(X)

In [ ]:
scaled_data = sc.transform(X)
scaled_data

### Implementasi PCA

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=3)
 
pca_features = pca.fit_transform(scaled_data)
 
# print('Shape before PCA: ', x_scaled.shape)
# print('Shape after PCA: ', pca_features.shape)
 
pca_df = pd.DataFrame(
    data=pca_features, 
    columns=['PC1','PC2','PC3'])

In [ ]:
data = pd.concat([pca_df,Y],axis=1)
data

In [ ]:
features = ['PC1', 'PC2', 'PC3']
# Separating out the features
X = data.loc[:, features].values
# Separating out the target
Y = data.loc[:,['Label']].values

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
 X, Y, test_size = 0.2, random_state = 100)
y_train=y_train.ravel()
y_test=y_test.ravel()

## Implementasi metode Naive Bayes

## - Naive Bayes adalah metode klasifikasi data berdasarkan probabilitas yang bisa saja terjadi di masa depan. Metode ini dianggap simpel dan efektif untuk diterapkan dalam analisis perusahaan.

## Implementasi metode KNN


## - KNN merupakan algoritma klasifikasi yang paling sederhana dalam mengklasifikasikan sebuah gambar kedalam sebuah label. Metode ini mudah dipahami dibandingkan metode lain karena mengklasifikasikan berdasarkan jarak terdekat dengan objek lain (tetangga).

## Word Embedding Word2vec CBOW

In [ ]:
import gensim
from gensim.models import Word2Vec

In [ ]:
# Create CBOW model
model1 = gensim.models.Word2Vec(df['stemming'], min_count = 1,
                              vector_size = 100, window = 5)
 
# Print results
print("Cosine similarity between 'teknologi' " +
               "and 'system' - CBOW : ",
    model1.wv.similarity('teknologi', 'system'))
     
print("Cosine similarity between 'teknologi' " +
                 "and 'mobile' - CBOW : ",
      model1.wv.similarity('teknologi', 'mobile'))

In [ ]:
#getting similar word
sim_words = model1.wv.most_similar('teknologi')
sim_words